# tScanner
A RSA Public Key Scanner

## Web Mining Final Project
### Daiwei Chen

This project is to go alongside of [Tetanus](https://github.com/ForeverAnApple/Tetanus), the Batch GCD RSA Weak Keys cracker. The purpose of this tool is to obtain public keys from SSL certificates (SSH, PGP may come later). The other point of this tool is to format the RSA keys correctly to extract the modulus of each key to insert into Tetanus for further analysis.

# Certificate Scanning Stage
First, a random amount of IPs will be generated in order to attempt a port 443 SSL handshake to obtain the server certificate. The certificates will be stored for furture public key and public key modulus extraction at a later stage.

During the certificate scanning stage, this slice of code will attempt to perform a socket connection to the given ip. With a default timeout of 10 seconds. Since all the ips are randomly generated, we will not be sure that the IP has a running webserver on 443 with HTTPS. The code will reflect on that and continue to test new generated IPs until it reaches the VALID_CERTS limit.

In [ ]:
import ssl
import time
import random
import pickle
import os
from socket import *
from cryptography import x509
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.serialization import PublicFormat
from cryptography.hazmat.primitives.serialization import Encoding
from random import getrandbits
from ipaddress import IPv4Address

random.seed(time.time())

# Load the dictionary, if possible
if os.path.exists(os.path.join(os.getcwd(), 'scanned_certs.pkl')):
    with open('scanned_certs.pkl', 'rb') as f:
        certs = pickle.load(f)
else:
    certs = dict()

# Load the set of already tried IPs
if os.path.exists(os.path.join(os.getcwd(), 'scanned_ips.pkl')):
    with open('scanned_ips.pkl', 'rb') as f:
        scannedIps = pickle.load(f)
else:
    scannedIps = set()

print("Loaded", len(certs), "certificate(s) and", len(scannedIps), "scanned IPs.")

VALID_CERTS = 100
print("Attempting to scan for", VALID_CERTS, "valid certificate(s).")

setdefaulttimeout(2) # time is of the essence

i = len(certs)
tries = 0
successful = 0

start = time.time()

# Randomly Generate IPs and attempt to obtain a valid SSL 
# certificate off of them
while i < VALID_CERTS:
    tries += 1
    bits = getrandbits(32)
    addr = IPv4Address(bits)
    addr_str = str(addr)
    #print("Trying", addr_str)
    if addr_str not in scannedIps:
        scannedIps.add(addr_str)
        try:                                                                                                              
          cert = ssl.get_server_certificate((addr_str, 443))
        except (error, timeout) as err:
          cert = "Timed Out"
        
        if cert != "Timed Out":
            certs[addr_str] = cert
            #print("Certificate extracted from", addr_str)
            successful += 1
            i += 1
        else:
            pass
            #print(addr_str, "timed out")
            
#print(certs)
print("Tried", tries, "ips for", successful, "successful certificate extractions.")
end = time.time()
print("Scanning for", successful, "valid certificates took", (end-start)/60, "minutes")
with open("scanned_certs.pkl", 'wb') as f:
    pickle.dump(certs, f)
    
with open("scanned_ips.pkl", 'wb') as f:
    pickle.dump(scannedIps, f)

Loaded 1 certificates and 3 scanned IPs.
Attempting to scan for 100 valid certificate(s).


# Public Key Extraction
Next, we proceed to extract the public key from the certificate, as well as extracting the public components n (modulus) and e. This information will be saved onto a text file in hex form as to send to Tetanus for furthur analysis.

In [ ]:
# Load the dictionary, if possible
if os.path.exists(os.path.join(os.getcwd(), 'scanned_certs.pkl')):
    with open('scanned_certs.pkl', 'rb') as f:
        certs = pickle.load(f)
else:
    print("Certificates pickle missing!")
    exit()
    
print("Loaded", len(certs), "certificate(s) for public key info extraction.")

key = x509.load_pem_x509_certificate(cert.encode(), default_backend())
keyPub = key.public_key().public_bytes(Encoding.PEM, PublicFormat.SubjectPublicKeyInfo)

# Analyzing and Graphing
After the modulus(es) information is given to Tetanus to perform Batch-GCD attacks on. Next, tScanner will attempt to produce graphs to represent the information in piechart form.